# JWebbinar JDAT Demonstration of Mosviz
Welcome to JWST Data Analysis Tools (JDAT) demonstration of Mosviz, a Multi-Object Spectrograph visualziation tool built atop the Jdaviz framework. This notebook will demonstrate some of the features we've built into Mosviz to facilitate the specialized usecase of MOS analysis. To learn more about Mosviz, see our [Mosviz Documentation](https://jdaviz.readthedocs.io/en/latest/mosviz/index.html). To learn more about the Jdaviz framework and how you can build your own tool for your usecase, see our [Jdaviz Documentation](https://jdaviz.readthedocs.io/)

Mosviz is specifically designed around processing NIRISS Wide Field Slitless Spectroscopy (WFSS) and NIRSpec MOS Mode on JWST. This demo goes through a simulated NIRISS WFSS MOS dataset and demonstrates the MOS-specific features of Mosviz to help visualize and analyze your data. Specifically, in this demo, we will attempt to identify a spectral emission line and model fit the emission curve:
1. Start Mosviz
2. Download and load data file
3. Select Source 6
4. Adjust scaling of 2D viewer identify spectral emission
5. Identify spectral emission in 1D spectrum viewer
6. Attempt to model fit the spectral emission line

## Starting Mosviz
First, we'll load up Mosviz and display the window. This window will initially be empty since we haven't loaded our data just yet

In [ ]:
from jdaviz import MosViz
mosviz = MosViz()
mosviz.app

## Downloading data
Now, we'll download our NIRISS WFSS simulation dataset from Box. Traditionally, NIRISS WFSS datasets will contain both Column and Row dispersions. For brevity purposes, this dataset has stripped out the Row dispersions and only contains the Column dispersions. We'll expect Mosviz to warn us about this at when we load it. 

In [ ]:
# Download file
from astropy.utils.data import download_file
data_archive_path = download_file("https://stsci.box.com/shared/static/bij29ciuxxpb2sdhnh4j2q3np1lq9yxr.zip")
# Unzip file
import tempfile
niriss_folder = tempfile.mkdtemp()
print(niriss_folder)
from zipfile import ZipFile
with ZipFile(data_archive_path, 'r') as data_archive:
    data_archive.extractall(niriss_folder)

## Load data into Mosviz
We have developed a specific parser for NIRISS data for Mosviz. This handles all the instrument specific parsing required to visualize NIRISS WFSS data. Once you see `Done` printed, the the dataset has been successfully loaded into Mosviz, and you should see the table populate with all the different cutouts above. Note again this dataset has removed the Row dispersions for brevity, so you will see some warning messages about those missing files during load time. This is expected for this demonstration.

NOTE: At the time of writing this notebook, our team is aware of a `IndexError: list index out of range` traceback at loadtime. You can safely ignore this traceback, as it has been tested not to impact the current behavior of the tool. We are working to resolve this issue, and we appreciate your patience!

NOTE: At the time of writing this notebook, our team is aware of a `ValueError: FITS header values must contain standard printable ASCII character` traceback when selecting rows in the table viewer. You can safely ignore this traceback, as it has been tested not to impact the current behavior of the tool. We are working to resolve this issue, and we appreciate your patience!

In [ ]:
# Load extracted data into Mosviz
mosviz.load_niriss_data(niriss_folder)

The remainder of the demo will occur within the Mosviz App being displayed in Cell 2. 

## UI Tasks
### Select Target 6
The Mosviz Table Viewer displays all of the different dispersed spectra cutouts for each target identified in the field of view. To switch between targets, you can click on the corresponding row. For this exercise, we'll be looking at "Source ID 6"

### Visually identify the emission feature
You can find an assortment of tools in the Toolbox icon in the upper-left corner of each viewer. Amongst them you can find a series of pan/zoom tools, including a bidirectional and individual axis panning and zooming. Using these tools, zoom into the spectral feature around 1.4um in the 1D spectral viewer. As you zoom around, you will notice the first-of-many MOS features we've built into Mosviz: linked data viewers. As the field of view is adjusted in either the 1D or 2D spectral viewer, the other viewer will synchronously pan and zoom as well.

If you're having issues zooming, you can use the following cell to programmatically zoom into the region of interest after you have selected Source ID 6. You should see both spectral viewers synchronously zoom into the same region, even though this command only sets the ranges for the 1D Spectrum Viewer

In [ ]:
scale_x = mosviz.app.get_viewer("spectrum-viewer").scale_x
scale_x.min = 1.35
scale_x.max = 1.6

scale_y = mosviz.app.get_viewer("spectrum-viewer").scale_y
scale_y.min = 0.8
scale_y.max = 2.2

To make the feature more visible in the 2D spectral viewer, we'll need to adjust the stretch to the correct levels. To do this, open the Toolbox and open the" plot options by pressing the "Gear" icon. From there, you can adjust how your data looks by clicking the "Layer" tab. Modify the "min" value to 0 and the "max" value to 10.

### Attempt to fit the Spectral Emission
Many of the features you've seen in other the other tools built on the Jdaviz framework are automatically included here in Mosviz as well. We'll be demonstrating one of them here: the Model Fitting Plugin. This plugin is designed to attempt to fit a model onto a 1D spectral viewer. You can find our plugin tray on the right side of Jdaviz under the "Plugin" icon in the top right corner. From there, select the `Model Fitting` dropdown. We will be attempting a simple Constant + Gaussian model fit.
1. To start, we'll need to select the spectral emission feature we want to fit. To do this, open the toolbar in the 1D spectral viewer and select the "Region Select" tool. Once we've zoomed in to visually identify the emission feature, use your mouse to draw a region around the spectral feature.
2. With the region selected, you can inspect your "Regions Tray" at the top of the viewer. Here you'll see an inventory of all the regions you've drawn. If this is the first region you've selected, it should be named, by default, as "Subset 1". 
3. In the Model Fitting Plugin, select "Subset 1" as the data we'll be fitting against. 
4. Add a constant model and give it a name, in this case "Constant". This name should not include any whitespaces (spaces, tabs, etc.). You should see a constant model being added with an initial "best fit" provided by the tool.
5. Likewise add a gaussian model and give it a name, in this case "Gaussian".
6. In the "Model Equation Editor", type out your equation. In this case, we'll be attempting a simple Constant + Gaussian model fit, so I'll enter `Constant + Gaussian`. You'll substitute the names with the ones you've provided above.
7. Hit Fit and a fit will be attempted on the spectrum you've selected. Based on your selected region, our backend will attempt to fit your model as best it can. If you get unsatisfactory results, try redrawing your model and trying again.

This concludes the Jwebbinar demonstration of Mosviz. There are many other features and plugins we've been developing into the tool, so we encourage you to play around and try things.

To explore more about Jdaviz, and the other tools we've developed, you can learn more at our [Jdaviz Documentation](https://jdaviz.readthedocs.io/), or our [Github Homepage](https://github.com/spacetelescope/jdaviz). To learn more about Mosviz specifically, see our [Mosviz Documentation](https://jdaviz.readthedocs.io/en/latest/mosviz/index.html).

If you run into issues, or have a new feature request, we'd love to hear about them. You can make these requests, or report any issues you've found, [here](https://github.com/spacetelescope/jdaviz/issues/new/choose). We encourage you to create a Github account and collaborate with us so that we can be sure we've addressed all of your concerns. Otherwise, you can contact any of our team members, or submit a JWST Help Desk ticket, and we'll do our best to help!